In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
# analog_file = r"C:\Users\neurogears\Documents\git\neurogears\sarahe-vr-2p\analysis\data\AnalogInput.bin"
analog_file = r"C:\\Mouse2\\2023-07-20T16-00-43\AnalogInput.bin"
fs_analog = 1000 #samples per second
digital_file =r"C:\\Mouse2\\2023-07-20T16-00-43\DigitalInput.bin"
fs_digital = 3000 #samples per second

In [323]:
analog_data = np.fromfile(analog_file, dtype=np.float64).reshape(-1,3).transpose()
for i in range(analog_data.shape[0]): 
    plt.figure()
    plt.plot(analog_data[i,:])
    plt.show()

analog_piezo = analog_data[0,:] # specify input
analog_time = np.arange(analog_piezo.shape[0])/fs_analog #change time scale
digital_time = np.arange(digital_data.shape[0])/fs_digital

# analog_piezo[np.searchsorted(analog_time,digital_time)]

plot figure
plt.figure(figsize=(20,10))
plt.plot(analog_time, analog_photodiode)
plt.scatter(rising_edges / fs_analog, rising_edges*0 + 1, c='r', s=10)
plt.xlim((0, 5))
plt.show()


IndexError: index 52100 is out of bounds for axis 0 with size 52100

In [42]:

def extract_digital_channel(digital_data, channel_number):
     #channel_number goes from 0 to 7
     return digital_data & (1 << channel_number) > 0

In [321]:
digital_data = np.fromfile(digital_file, dtype=np.byte)
digital_time = np.arange(digital_data.shape[0])/fs_digital

def detect_rising_edges(signal):
    edges = np.where((np.diff(signal) > 0) & (signal[1:] == 1))[0] + 1
    return edges

photodiode_signal = extract_digital_channel(digital_data, 2)
diode_rising_edges = detect_rising_edges(photodiode_signal)
# rising_edges = np.where(np.diff(extract_digital_channel(digital_data, 2)) == 1)[0] +1# identify where in time frame signal
scanner_signal= np.logical_not(extract_digital_channel(digital_data,0))
scanner_edges = detect_rising_edges(scanner_signal)
frame_indices = np.arange(1, len(diode_rising_edges) + 1)
scanner_frame_indices=np.arange(1, len(scanner_edges) + 1)
aligned_scanner=np.searchsorted(scanner_frame_indices,frame_indices)

plt.figure()
# plt.plot(digital_time, photodiode_signal)
# plt.plot(digital_time,scanner_signal)
# plt.scatter(rising_edges / fs_digital, photodiode_signal[rising_edges], c='r', s=10)
# plt.scatter(scanner_edges / fs_digital, scanner_signal[scanner_edges], c='r', s=10)
# plt.xlim(10, 10.1)
plt.show()



<Figure size 640x480 with 0 Axes>

In [310]:
# read and align rotary encoder values
csv_file = r"C:\\Mouse2\\2023-07-20T16-00-43\RotaryEncoder.csv"  # add position of encoder to aligned data array
csv_data = pd.read_csv(csv_file)
csv_frame_indices=csv_data['FrameIndex'].values + 1
csv_values=csv_data['RotaryEncoder'].values
aligned_encoder_values = np.full(len(frame_indices), np.nan)
aligned_encoder_values[csv_frame_indices - 1] = csv_values


csv_file = r"C:\\Mouse2\\2023-07-20T16-00-43\VrPosition.csv"  # add position of encoder to aligned data array
csv_data = pd.read_csv(csv_file)
csv_values=csv_data['Position'].values
# aligned_position_values = np.full(len(frame_indices), np.nan)
# aligned_position_values[csv_frame_indices - 1] = csv_values# what frames are these corresponding to?

csv_file = r"C:\\Mouse2\\2023-07-20T16-00-43\FrameEvents.csv"  # add position of encoder to aligned data array
csv_data = pd.read_csv(csv_file)
lick_index=np.where(csv_data['EventName'] == 'Lick')[0]
lick_frame_indices = csv_data['Frame.Index'].values[lick_index]
aligned_licks = np.zeros(len(frame_indices), dtype=int)
aligned_licks[np.searchsorted(frame_indices, lick_frame_indices)] = 1

teleport_index=np.where(csv_data['EventName'] == 'Teleport')[0]
teleport_frame_indices = csv_data['Frame.Index'].values[teleport_index]
aligned_teleport = np.zeros(len(frame_indices), dtype=int)
aligned_teleport[np.searchsorted(frame_indices, teleport_frame_indices)] = 1

reward_index=np.where(csv_data['EventName'] == 'Reward')[0]

reward_frame_indices = csv_data['Frame.Index'].values[reward_index]
aligned_reward = np.zeros(len(frame_indices), dtype=int)
aligned_reward[np.searchsorted(frame_indices, reward_frame_indices)] = 1

In [311]:
data = {
    'FrameIndex': frame_indices,
    'Scanner': aligned_scanner,
    'Encoder': aligned_encoder_values,
    'Lick': aligned_licks,
    'Reward': aligned_reward,
    'Teleport': aligned_teleport
}

behavior_data = pd.DataFrame(data)

print(behavior_data)

       FrameIndex  Scanner  Encoder  Lick  Reward  Teleport
0               1        0      0.0     0       0         0
1               2        1      0.0     0       0         0
2               3        2      0.0     0       0         0
3               4        3      0.0     0       0         0
4               5        4      0.0     0       0         0
...           ...      ...      ...   ...     ...       ...
20318       20319     1582      NaN     0       0         0
20319       20320     1582      NaN     0       0         0
20320       20321     1582      NaN     0       0         0
20321       20322     1582      NaN     0       0         0
20322       20323     1582      NaN     0       0         0

[20323 rows x 6 columns]


In [80]:
## align and concatenate all digital and analog data

aligned_data = np.zeros(len(digital_time), dtype=[('Timestamp', float), ('Analog_Data0', float), ('Analog_Data1', float), ('Analog_Data2', float),
                                                  ('Digital_Data0', int), ('Digital_Data1', int), ('Digital_Data2', int),('RotaryEncoder',float),('EventNames',str)])
aligned_data['Timestamp'] = digital_time
for i in range(len(analog_data)):
    aligned_data['Analog_Data'+ str(i)] = analog_data[i,:][np.searchsorted(analog_time,digital_time)]
    aligned_data['Digital_Data'+str(i)]= extract_digital_channel(digital_data, 2*i)  
# plt.figure()
# plt.plot(aligned_data['Timestamp'],aligned_data['Digital_Data0'], c = "b")
# plt.plot(aligned_data['Timestamp'],aligned_data['Analog_Data2'], c = "r")
# plt.show()


156300
20323
156300


IndexError: index 52100 is out of bounds for axis 0 with size 52100